In [1]:
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import datetime
import random

page = set()
random.seed(datetime.datetime.now())

C:\Users\a\AppData\Local\Temp/ipykernel_4952/1370226561.py:9: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(datetime.datetime.now())


In [9]:
# 获取页面中所有的内链链表
def getInternalLinks(bs, includeUrl):
    includeUrl = '{}://{}'.format(urlparse(includeUrl).scheme, urlparse(includeUrl).netloc)
    internalLinks = []

    # 找出所有以"/"开头的链接
    for link in bs.find_all('a', href=re.compile('^(/|.*' + includeUrl + ')')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                if link.attrs['href'].startswith('/'):
                    internalLinks.append(includeUrl + link.attrs['href'])
                else:
                    internalLinks.append(link.attrs['href'])

    return internalLinks

In [3]:
# 获取页面中所有外链的列表
def getExternalLinks(bs, excludeUrl):
    externalLinks = []

    # 找出所有以"http"或"www"开头且不包含当前URL的链接
    for link in bs.find_all('a', href=re.compile('^(http|www)((?!' + excludeUrl + ').)*$')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])

    return externalLinks

In [4]:
# 随机获取一个外链
def getRandomExternalLink(startingPage):
    html = urlopen(startingPage)
    bs = BeautifulSoup(html, 'html.parser')
    externalLinks = getExternalLinks(bs, urlparse(startingPage).netloc)
    if len(externalLinks) == 0:
        print("No external links, looking around the site for one")
        domain = f"{urlparse(startingPage).scheme}://{urlparse(startingPage).netloc}"
        internalLinks = getInternalLinks(bs, domain)
        return getRandomExternalLink(internalLinks[random.randint(0, len(internalLinks) - 1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks) - 1)]

In [5]:
def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print(f"Random external link is {externalLink}")
    followExternalOnly(externalLink)

In [6]:
html = urlopen("http://oreilly.com")
bs = BeautifulSoup(html, 'html.parser')
links = bs.find_all('a')
links

[<a href="#maincontent"><span class="skipToMain-text">Skip to main content</span></a>,
 <a aria-current="page" class="logo" href="https://www.oreilly.com" title="home page"><img alt="O'Reilly home" onerror="this.src='https://cdn.oreillystatic.com/images/sitewide-headers/oreilly_logo_mark_red_@2x.png'; this.onerror=null;" src="https://cdn.oreillystatic.com/images/sitewide-headers/oreilly_logo_mark_red.svg"/></a>,
 <a href="https://learning.oreilly.com/accounts/login-check/">Sign In</a>,
 <a class="menuList-cta" href="https://www.oreilly.com/online-learning/try-now.html" id="nav-tryNow">Try Now</a>,
 <a href="https://www.oreilly.com/online-learning/teams.html">Teams</a>,
 <a href="https://www.oreilly.com/online-learning/teams.html">For business</a>,
 <a href="https://www.oreilly.com/online-learning/government.html">For government</a>,
 <a href="https://www.oreilly.com/online-learning/academic.html">For higher ed</a>,
 <a href="https://www.oreilly.com/online-learning/individuals.html">Ind

In [10]:
links = followExternalOnly("http://oreilly.com")
links

Random external link is https://twitter.com/oreillymedia
No external links, looking around the site for one
Random external link is https://cdn.cms-twdigitalassets.com/content/dam/legal-twitter/site-assets/tos-aug-19th-2021/Twitter_User_Agreement_EN.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


No external links, looking around the site for one


ValueError: empty range for randrange() (0, 0, 0)